In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from tqdm import tqdm
from scipy.stats import gaussian_kde
import os

torch.manual_seed(42)

# Parameters for different Gaussian distributions
num_samples = 1000
mean_1, std_1 = 0, 1
mean_2, std_2 = 5, 2
mean_3, std_3 = -3, 0.5

# Generate random samples
gaussian_data_1 = torch.normal(mean_1, std_1, size=(num_samples,))
gaussian_data_2 = torch.normal(mean_2, std_2, size=(num_samples,))
gaussian_data_3 = torch.normal(mean_3, std_3, size=(num_samples,))
x_0 = torch.cat([gaussian_data_1, gaussian_data_2, gaussian_data_3])

# Beta scheduler functions
def linear_beta_schedule(T, beta_min, beta_max):
    return torch.linspace(beta_min, beta_max, T)

def constant_beta_schedule(T, beta_value):
    return torch.full((T,), beta_value)

def exponential_beta_schedule(T, beta_min, beta_max):
    return torch.exp(torch.linspace(torch.log(torch.tensor(beta_min)), torch.log(torch.tensor(beta_max)), T))

# KL divergence calculation function
def calculate_kl_divergence(true_dist, approx_dist):
    kde_true = gaussian_kde(true_dist, bw_method='silverman')
    kde_approx = gaussian_kde(approx_dist, bw_method='silverman')
    x_grid = np.linspace(min(true_dist.min(), approx_dist.min()), max(true_dist.max(), approx_dist.max()), 1000)
    p_true = kde_true(x_grid)
    p_approx = kde_approx(x_grid)
    p_true = p_true / np.sum(p_true)
    p_approx = p_approx / np.sum(p_approx)
    return np.sum(p_true * np.log(p_true / p_approx))

# Forward diffusion function
def forward_diffusion(x_0, betas, T):
    sqrt_one_minus_betas = torch.sqrt(1 - betas)
    sqrt_betas = torch.sqrt(betas)
    x_t = x_0.clone()
    trajectory = [x_t]
    for t in range(T):
        noise = torch.randn_like(x_t)
        x_t = sqrt_one_minus_betas[t] * x_t + sqrt_betas[t] * noise
        trajectory.append(x_t)
    return trajectory

# Reverse denoising function
def reverse_denoising2(trajectory, betas, T, min_beta_tilde=1e-4, noise_floor=1e-4):
    alphas = 1 - betas
    alpha_bar = torch.cumprod(alphas, dim=0)
    xt = trajectory[-1]
    reverse_trajectory = [xt]
    x_0_trajectory = trajectory[0]
    for t in tqdm(reversed(range(1, T))):
        alpha_bar_t_1 = alpha_bar[t - 1]
        alpha_bar_t = alpha_bar[t]
        beta_t = betas[t - 1]
        sqrt_alpha_bar_t_1 = torch.sqrt(alpha_bar_t_1)
        sqrt_alpha_bar_t = torch.sqrt(alpha_bar_t)
        one_minus_alpha_bar_t = 1 - alpha_bar_t
        one_minus_alpha_bar_t_1 = 1 - alpha_bar_t_1
        mu_t = (sqrt_alpha_bar_t_1 / one_minus_alpha_bar_t) * beta_t * x_0 + (torch.sqrt(alphas)[t] * one_minus_alpha_bar_t_1 / one_minus_alpha_bar_t) * xt
        # beta_t_tilde = torch.maximum((1 - alpha_bar_t_1) / (1 - alpha_bar_t) * beta_t, torch.tensor(min_beta_tilde))
        beta_t_tilde = (1 - alpha_bar_t_1) / (1 - alpha_bar_t) * beta_t
        noise = torch.randn_like(xt) * torch.sqrt(beta_t_tilde + noise_floor)
        # print('----------')
        # print('t:', t)
        # print(f'beta_t_tilde: {beta_t_tilde}')
        # print(f' nosie : {noise}')
        # print(f'mu_t: {mu_t}')
        # print(f' xt: {xt}')
        # print(f' x_0: {x_0}')
        # print(f' coeff x0: {(sqrt_alpha_bar_t_1 / one_minus_alpha_bar_t) * beta_t }')
        # print(f'      sqrt_alpha_bar_t_1: {sqrt_alpha_bar_t_1}')
        # print(f'      one_minus_alpha_bar_t: {one_minus_alpha_bar_t}')
        # print(f'      beta_t: {beta_t}')
        # print(f' coeff xt: {(torch.sqrt(alphas)[t] * one_minus_alpha_bar_t_1 / one_minus_alpha_bar_t)}')
        # print(f'      sqrt(alphas)[t]: {torch.sqrt(alphas)[t]}')
        # print(f'      one_minus_alpha_bar_t_1: {one_minus_alpha_bar_t_1}')
        # print(f'      one_minus_alpha_bar_t: {one_minus_alpha_bar_t}')
        # print(f'----------------------')
        xt = mu_t + noise
        reverse_trajectory.append(xt)
    return reverse_trajectory

# Update function for the forward diffusion plot
def update_plot1(frame, trajectory, x_0, ax):
    ax.clear()
    kl_divergence = calculate_kl_divergence(x_0.numpy(), trajectory[frame].numpy())
    ax.hist(trajectory[frame].numpy(), bins=50, density=True, alpha=0.6, label=f"KL Divergence: {kl_divergence:.4f}")
    ax.hist(x_0.numpy(), bins=50, density=True, alpha=0.2, color='gray', label="Original Distribution")
    ax.set_title(f"q(x{frame}) - KL Divergence: {kl_divergence:.4f}")
    ax.set_xlim(-10, 10)
    ax.set_ylim(0, 0.4)
    ax.set_xlabel('x')
    ax.set_ylabel('Density')
    ax.legend()

# Update function for the reverse denoising plot
def update_plot2(frame, reverse_trajectory, x_0, ax):
    ax.clear()
    kl_divergence = calculate_kl_divergence(x_0.numpy(), reverse_trajectory[frame].numpy())
    ax.hist(reverse_trajectory[frame].numpy(), bins=50, density=True, alpha=0.6, label=f"KL Divergence: {kl_divergence:.4f}")
    ax.hist(x_0.numpy(), bins=50, density=True, alpha=0.2, color='gray', label="Original Distribution")
    ax.set_title(f"Reverse Step {T - frame} - KL Divergence: {kl_divergence:.4f}")
    ax.set_xlim(-10, 10)
    ax.set_ylim(0, 0.4)
    ax.set_xlabel('x')
    ax.set_ylabel('Density')
    ax.legend()

# Main function to run experiments
def run_experiment(T, beta_scheduler, beta_params, result_folder, scheduler_name):
    os.makedirs(result_folder, exist_ok=True)
    
    if beta_scheduler == "linear":
        betas = linear_beta_schedule(T, **beta_params)
        beta_info=f'beta_range_{beta_params["beta_min"]}_{beta_params["beta_max"]}'
    elif beta_scheduler == "constant":
        betas = constant_beta_schedule(T, **beta_params)
        beta_info=f'beta_{beta_params["beta_value"]}'
    elif beta_scheduler == "exponential":
        betas = exponential_beta_schedule(T, **beta_params)
        beta_info=f'beta_range_{beta_params["beta_min"]}_{beta_params["beta_max"]}'
    else:
        raise ValueError("Invalid beta scheduler")

    trajectory = forward_diffusion(x_0, betas, T)
    print("Forward process completed. Running reverse denoising...")
    reverse_trajectory = reverse_denoising2(trajectory, betas, T)
    print("Reverse denoising completed.")
    frames_to_plot = range(0, T, max(1, T // 100))
    #make sure the last and first 10 frames are included with no step_size
    frames_to_plot =  list(range(1, T // 100))+ list(frames_to_plot) + list(range(T-T // 100, T-1)) 
    print(f"Frames to plot: {frames_to_plot}")
     # Calculate the interval for 5 seconds
    total_duration = 5000  # 5 seconds in milliseconds
    interval = total_duration / T  # Time between frames in milliseconds
    
    # Forward diffusion animation
    fig, ax = plt.subplots(figsize=(6, 4))
    ani = animation.FuncAnimation(fig, update_plot1, frames=frames_to_plot, interval=interval, repeat=False, fargs=(trajectory, x_0, ax))
    ani.save(os.path.join(result_folder, f'forward_{scheduler_name}_timesteps_{T}_beta_{beta_info}.gif'), writer='imagemagick')
    plt.close(fig)
    print("Forward diffusion animation saved.")
    
    # Reverse denoising animation
    fig, ax = plt.subplots(figsize=(6, 4))
    ani = animation.FuncAnimation(fig, update_plot2, frames=frames_to_plot, interval=interval, repeat=False, fargs=(reverse_trajectory, x_0, ax))
    ani.save(os.path.join(result_folder, f'reverse_diffusion_scheduler_{scheduler_name}_timesteps_{T}_beta_{beta_info}.gif'), writer='imagemagick')
    plt.close(fig)
    print("Reverse denoising animation saved.")

# Example usage
T = 5000
result_folder = "results"
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "linear")
# run_experiment(T, "constant", {"beta_value": 0.01}, result_folder, "constant")
# run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "exponential")


Forward process completed. Running reverse denoising...


4999it [00:01, 4932.37it/s]


Reverse denoising completed.
Frames to plot: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 2850, 2900, 2950, 3000, 3050, 3100, 3150, 3200, 3250, 3300, 3350, 3400, 3450, 3500, 3550, 3600, 3650, 3700, 3750, 3800, 3850, 3900, 3950, 4000, 4050, 4100, 4150, 4200, 4250, 4300, 4350, 4400, 4450, 4500, 4550, 4600, 4650, 4700, 4750, 4800, 4850, 4900, 4950, 4950, 4951, 4952, 4953, 4954, 4955, 4956, 4957, 4958, 4959, 4960, 4961, 4962, 4963, 4964, 4965, 4966, 4967, 4968, 4969, 4970, 4971, 4972, 4973, 4974, 4975, 4976, 4977, 4978, 4979, 4980, 4981

MovieWriter imagemagick unavailable; using Pillow instead.
/tmp/ipykernel_1501/286780614.py:42: RuntimeWarning: overflow encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
/tmp/ipykernel_1501/286780614.py:42: RuntimeWarning: divide by zero encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
MovieWriter imagemagick unavailable; using Pillow instead.


Forward diffusion animation saved.


/tmp/ipykernel_1501/286780614.py:42: RuntimeWarning: overflow encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
/tmp/ipykernel_1501/286780614.py:42: RuntimeWarning: divide by zero encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))


Reverse denoising animation saved.


In [9]:
T = 500
result_folder = "results"
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.01}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "exponential")

T = 100
result_folder = "results"
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.01}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "exponential")

Forward process completed. Running reverse denoising...


0it [00:00, ?it/s]

499it [00:00, 3666.38it/s]

Reverse denoising completed.
Frames to plot: [1, 2, 3, 4, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390, 395, 400, 405, 410, 415, 420, 425, 430, 435, 440, 445, 450, 455, 460, 465, 470, 475, 480, 485, 490, 495, 495, 496, 497, 498]



MovieWriter imagemagick unavailable; using Pillow instead.
/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: overflow encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: divide by zero encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
MovieWriter imagemagick unavailable; using Pillow instead.


...

Forward diffusion animation saved.
Reverse denoising animation saved.
Forward process completed. Running reverse denoising...


99it [00:00, 4646.62it/s]
MovieWriter imagemagick unavailable; using Pillow instead.


Reverse denoising completed.
Frames to plot: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


MovieWriter imagemagick unavailable; using Pillow instead.


Forward diffusion animation saved.
Reverse denoising animation saved.


In [10]:
T = 20
result_folder = "results"
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "exponential")
T = 10
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "exponential")
T = 50
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "exponential")


Forward process completed. Running reverse denoising...


19it [00:00, 5862.71it/s]
MovieWriter imagemagick unavailable; using Pillow instead.


Reverse denoising completed.
Frames to plot: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: divide by zero encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: overflow encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
MovieWriter imagemagick unavailable; using Pillow instead.


Forward diffusion animation saved.


...

Forward diffusion animation saved.
Reverse denoising animation saved.
Forward process completed. Running reverse denoising...


49it [00:00, 6395.35it/s]
MovieWriter imagemagick unavailable; using Pillow instead.


Reverse denoising completed.
Frames to plot: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


MovieWriter imagemagick unavailable; using Pillow instead.


Forward diffusion animation saved.
Reverse denoising animation saved.


In [11]:
T = 100
result_folder = "results_beta_0_01"
run_experiment(T, "linear", {"beta_min": 0.01, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.01, "beta_max": 0.5}, result_folder, "exponential")
T = 20
result_folder = "results_beta_0_01"
run_experiment(T, "linear", {"beta_min": 0.01, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.01, "beta_max": 0.5}, result_folder, "exponential")
T = 10
result_folder = "results_beta_0_01"
run_experiment(T, "linear", {"beta_min": 0.01, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.01, "beta_max": 0.5}, result_folder, "exponential")
T = 50
result_folder = "results_beta_0_01"
run_experiment(T, "linear", {"beta_min": 0.01, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.01, "beta_max": 0.5}, result_folder, "exponential")

Forward process completed. Running reverse denoising...


99it [00:00, 5725.18it/s]
MovieWriter imagemagick unavailable; using Pillow instead.


Reverse denoising completed.
Frames to plot: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: overflow encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: divide by zero encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
MovieWriter imagemagick unavailable; using Pillow instead.


Forward diffusion animation saved.


...

Reverse denoising completed.
Frames to plot: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


MovieWriter imagemagick unavailable; using Pillow instead.


Forward diffusion animation saved.


/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: divide by zero encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: overflow encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))


Reverse denoising animation saved.


In [12]:
# Now we make an already gaussian distribution 
mean_1, std_1 = 4, 3

x_0 = torch.normal(mean_1, std_1, size=(num_samples,))

result_folder = "results_gaussian"
T = 1000
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.01}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "exponential")
T = 500
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.01}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "exponential")
T = 100
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.01}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.1}, result_folder, "exponential")
T = 50
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "exponential")
T = 20
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "exponential")
T = 10
run_experiment(T, "linear", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "linear")
run_experiment(T, "constant", {"beta_value": 0.1}, result_folder, "constant")
run_experiment(T, "exponential", {"beta_min": 0.0001, "beta_max": 0.5}, result_folder, "exponential")


Forward process completed. Running reverse denoising...


999it [00:00, 5951.38it/s]

Reverse denoising completed.
Frames to plot: [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 990, 991, 992, 993, 994, 995, 996, 997, 998]



MovieWriter imagemagick unavailable; using Pillow instead.
/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: divide by zero encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
/tmp/ipykernel_213388/3996909593.py:42: RuntimeWarning: overflow encountered in divide
  return np.sum(p_true * np.log(p_true / p_approx))
MovieWriter imagemagick unavailable; using Pillow instead.


Forward diffusion animation saved.


...

Forward diffusion animation saved.
Reverse denoising animation saved.
Forward process completed. Running reverse denoising...


9it [00:00, 4038.59it/s]
MovieWriter imagemagick unavailable; using Pillow instead.


Reverse denoising completed.
Frames to plot: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


MovieWriter imagemagick unavailable; using Pillow instead.


Forward diffusion animation saved.
Reverse denoising animation saved.
